In [110]:
import tweepy
import pandas as pd
import yfinance as yf
import openai
import big_baller_moves
import json
import requests

In [111]:
#Put your Bearer Token in the parenthesis below
client1 = tweepy.Client(bearer_token=big_baller_moves.bossman_tingz["twitter_api_key"])

# Authenticate
auth = tweepy.OAuth1UserHandler(big_baller_moves.bossman_tingz["consumer_key"], big_baller_moves.bossman_tingz["consumer_secret"], big_baller_moves.bossman_tingz["access_token"], 
                                big_baller_moves.bossman_tingz["access_token_secret"])

# Create API object
api = tweepy.API(auth)

# openai key
openai.api_key = big_baller_moves.bossman_tingz["openai_api_key"]

In [140]:
#enter username to get the user ID
username = "Heisenberg_100k"

try:
    user = client1.get_user(username=username)
    user_id = user_id = user.data.id
    print("User ID for", username, ":", user_id)
except Exception as e:
    print("Error:", e)

User ID for Heisenberg_100k : 1750349614786691072


In [139]:
#get btc price (not working atm)

# Choose Token and Time
timestamp_open = pd.Timestamp('2024-05-02 05:09:00', tz='UTC')
unix_timestamp = int(timestamp_open.timestamp())
unix_timestamp_24 = unix_timestamp + (24*60*60)
unix_timestamp_48 = unix_timestamp + (48*60*60)
ckey = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp}&localization=false"
url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"
ckey24 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_24}&localization=false"
ckey48 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_48}&localization=false"

try:
        # Convert timestamp to Unix timestamp (Kraken API requires timestamp in seconds)
        unix_timestamp = int(timestamp.timestamp())

        # Kraken API endpoint for OHLC data
        url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"

response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'result' in data and 'XBTUSD' in data['result'] and data['result']['XBTUSD']:
                # The response contains a list of lists, where each inner list represents OHLC data for a specific interval
                # We extract the closing price from the last list (most recent data point)
                bitcoin_price = float(data['result']['XBTUSD'][-1][4])
                return bitcoin_price
            else:
                print("No data available for the specified timestamp")
                return None
        else:
            print(f"Failed to retrieve data from Kraken API. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


Failed to retrieve data from CoinGecko API


In [135]:
unix_timestamp

1714626540

In [141]:
#get all tweets
tweets = client1.get_users_tweets(
    id= user_id,
    max_results=100,  # Number of tweets to retrieve (adjust as needed)
    tweet_fields=['id', 'text', 'created_at'],  # Fields you want to retrieve for each tweet
    exclude = ['retweets','replies']
)

In [142]:
# Process retrieved tweets
for tweet in tweets.data:
    print(f"Tweet ID: {tweet.id}, Text: {tweet.text}, Created At: {tweet.created_at}")


Tweet ID: 1788936274507911189, Text: $SMMT Closed

In 4.82 (yesterday)

Out 4.96

+2.9%
+$182 https://t.co/FJwJ8S1RdE, Created At: 2024-05-10 14:16:59+00:00
Tweet ID: 1788662170185859172, Text: My $SPY call spreads expiring tomorrow are going to a 50/50 coin flip tomorrow. Had a chance to close at minimal loss today but chose not to. Bad move in hindsight.

Should I just let it ride?

Should I close it out tomorrow at a 60-70% loss?

This one and DIS were my two bad… https://t.co/1EwTM8bWPp https://t.co/HWpQ3uc0dx, Created At: 2024-05-09 20:07:47+00:00
Tweet ID: 1788649260701057334, Text: $GOOG Closed

In .67c credit (May 3rd)

Out .01c debit

+$330 profit https://t.co/shmUd4IV7w, Created At: 2024-05-09 19:16:29+00:00
Tweet ID: 1788601359010603103, Text: $BITX Closed

In 2.25 (today)

Out 2.80

+25% https://t.co/1FrUXsjpbc, Created At: 2024-05-09 16:06:09+00:00
Tweet ID: 1788579970635493749, Text: $ULTA Closed

In 5.6 (today)

Out 8.02

+44% https://t.co/FsZa8r2C56, Created At: 2024-05

In [143]:
df = pd.DataFrame(tweets.data)

In [144]:
df.head()

,created_at,edit_history_tweet_ids,id,text
0,2024-05-10 14:16:59+00:00,[1788936274507911189],1788936274507911189,$SMMT Closed\n\nIn 4.82 (yesterday)\n\nOut 4.96\n\n+2.9%\n+$182 https://t.co/FJwJ8S1RdE
1,2024-05-09 20:07:47+00:00,[1788662170185859172],1788662170185859172,My $SPY call spreads expiring tomorrow are going to a 50/50 coin flip tomorrow. Had a chance to close at minimal loss today but chose not to. Bad move in hindsight.\n\nShould I just let it ride?\n\nShould I close it out tomorrow at a 60-70% loss?\n\nThis one and DIS were my two bad… https://t.co/1EwTM8bWPp https://t.co/HWpQ3uc0dx
2,2024-05-09 19:16:29+00:00,[1788649260701057334],1788649260701057334,$GOOG Closed\n\nIn .67c credit (May 3rd)\n\nOut .01c debit\n\n+$330 profit https://t.co/shmUd4IV7w
3,2024-05-09 16:06:09+00:00,[1788601359010603103],1788601359010603103,$BITX Closed\n\nIn 2.25 (today)\n\nOut 2.80\n\n+25% https://t.co/1FrUXsjpbc
4,2024-05-09 14:41:09+00:00,[1788579970635493749],1788579970635493749,$ULTA Closed\n\nIn 5.6 (today)\n\nOut 8.02\n\n+44% https://t.co/FsZa8r2C56


In [41]:
ticker_symbol = 'AAPL'

# Fetch historical data
stock_data = yf.Ticker(ticker_symbol)

# Get historical market data
historical_data = stock_data.history(period="max")

print(historical_data)

                                 Open        High         Low       Close  \
Date                                                                        
1980-12-12 00:00:00-05:00    0.099192    0.099623    0.099192    0.099192   
1980-12-15 00:00:00-05:00    0.094448    0.094448    0.094017    0.094017   
1980-12-16 00:00:00-05:00    0.087548    0.087548    0.087117    0.087117   
1980-12-17 00:00:00-05:00    0.089273    0.089704    0.089273    0.089273   
1980-12-18 00:00:00-05:00    0.091861    0.092292    0.091861    0.091861   
...                               ...         ...         ...         ...   
2024-04-29 00:00:00-04:00  173.369995  176.029999  173.100006  173.500000   
2024-04-30 00:00:00-04:00  173.330002  174.990005  170.000000  170.330002   
2024-05-01 00:00:00-04:00  169.580002  172.710007  169.110001  169.300003   
2024-05-02 00:00:00-04:00  172.509995  173.419998  170.889999  173.029999   
2024-05-06 00:00:00-04:00  182.354004  184.169998  181.630005  181.835007   

In [75]:
client = openai
df1 = df.copy().head(5)
pd.set_option('display.max_colwidth', None)

In [76]:
#for astronomer
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [ ]:
#heisenberg
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [96]:
def get_response(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Tell me if the message is suggesting opening a long, short or N/A in one word. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short." }, 
                                                        {"role":"user","content": text}])
   
    return response.choices[0].message.content.strip()



In [106]:
def get_response_guided(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Your job is to tag tweets as either opening a long position or short position or N/A. Provide a reason explaining why the tweet is a long or short or n/a." }, 
                                                        {"role":"user","content": "Taking profit"},
                                                        {"role":"assistant","content": "Short, the poster is taking profit."}])
   
    return response.choices[0].message.content.strip()


In [ ]:
def get_response_image(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Your job is to tag tweets as either opening a long position or short position or N/A. Provide a reason explaining why the tweet is a long or short or n/a." }, 
                                                        {"role":"user","content": "Taking profit"},
                                                        {"role":"assistant","content": "Short, the poster is taking profit."}])
   
    return response.choices[0].message.content.strip()

In [107]:
df1['Direction (Guided)'] = df1['text'].apply(get_response_guided)
df1['Direction'] = df1['text'].apply(get_response)

In [108]:

selected = ['text','Direction','Direction (Guided)']
df1[selected]

,text,Direction,Direction (Guided)
0,"#Sentiment check\n\nWhich camp are you ? ⛺️\n\nA: Camp bulls, or \nB: Camp bears ?\n\nGive me your thoughts📊 https://t.co/8hyVQnk3tG",N/A,N/A. The tweet does not provide enough information to determine if the user is opening a long or short position.
1,"$BTC Update\n\nReceived the final squeeze we wanted🎯 prepare for downside\n\nIn last update, called how it's a good time to short around these prices but that weekend highs are likely to come first and how that's a good time to take profit of our big longs we have been building all… https://t.co/zGhswZAQ3r https://t.co/XFxkNagKE3 https://t.co/ma9kIVi8pR",Short,"N/A, as the tweet does not provide enough information to determine whether it is advocating for taking a long or short position."
2,"$BTC...\n\nCould smell that scam pump from a mile away \n\nSo figured was a good point to take advantage of the high prices.\n\nPin point tp. 🎯 Market dropping not one hour later.\n\nLike I said, still short, and back to 62k-60k zone.\n\n#BTC https://t.co/BrfvF8096U https://t.co/n6clGy0uhS",Short,"N/A, the tweet does not provide enough information to determine whether to open a long or short position."
3,"$BTC\n\nHitting it big, at our weekend highs\n\nSecond tp on our longs hit🎯\n\nTaking off another 25 % at 2.1 R 💰\n\nWe were building this position all week last week and doubled down as it moved to 57k because of our expectancy this range being bullish overall.\n\nPretty big position so… https://t.co/e4GnUA0JOJ https://t.co/Qw0K1GgrJH https://t.co/HetGlPXfai",long,"N/A, the tweet does not provide enough information to determine a clear long or short position."
4,"$BTC Update\n\nBullish weekly close, but don't be fooled, I'm still short\n\nLast update, stated how this range is bullish and how we're ultimately going to the upside, inevitably imo. Also talked about the possibility of a small pullback so took a short to decrease long risk and… https://t.co/qcz38nMoGG https://t.co/Ml20sLb2ZO https://t.co/iEmerXOtFG",Short,"N/A, the tweet does not provide enough information to determine if the poster is opening a long or short position."
